In [21]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Embedding
import tensorflow as tf

In [22]:
df = pd.read_csv("../data/transformed/wrangled.csv")

In [23]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [24]:
df.head()

,Unnamed: 0,id,title,author,text,label,nb_names,exclamation,text_length
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,17,0,10
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,12,0,9
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,11,0,16
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,4,0,7
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,3,0,2


In [25]:
len(df_train)

16608

In [26]:
len(df_test)

4153

Regression

In [27]:
model = tf.keras.Sequential([
      tf.keras.layers.Dense(32, activation='relu', input_shape=(3,)),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=[tf.keras.metrics.Accuracy()])

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                128       
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 1,217
Trainable params: 1,217
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(df[["nb_names", "exclamation", "text_length"]], df["label"], epochs=100, validation_split = 0.2)

Epoch 1/100
519/519 [==============================] - 1s 687us/step - loss: 0.3518 - accuracy: 0.0150 - val_loss: 0.3200 - val_accuracy: 0.0258
Epoch 2/100
519/519 [==============================] - 0s 533us/step - loss: 0.3250 - accuracy: 0.0509 - val_loss: 0.3166 - val_accuracy: 0.0508
Epoch 3/100
519/519 [==============================] - 0s 521us/step - loss: 0.3223 - accuracy: 0.0600 - val_loss: 0.3151 - val_accuracy: 0.0689
Epoch 4/100
519/519 [==============================] - 0s 531us/step - loss: 0.3217 - accuracy: 0.0798 - val_loss: 0.3189 - val_accuracy: 0.0727
Epoch 5/100
519/519 [==============================] - 0s 529us/step - loss: 0.3208 - accuracy: 0.0836 - val_loss: 0.3139 - val_accuracy: 0.0970
Epoch 6/100
519/519 [==============================] - 0s 531us/step - loss: 0.3203 - accuracy: 0.0918 - val_loss: 0.3144 - val_accuracy: 0.1096
Epoch 7/100
519/519 [==============================] - 0s 535us/step - loss: 0.3187 - accuracy: 0.1076 - val_loss: 0.3142 - val_ac

Tokenizer

In [30]:
# Shuffle data to improve performance
df_train = shuffle(df_train)

# Create list from data
text = []

for index, rows in df_train.iterrows(): 
    wordlist = str(rows["text"])
    text.append(wordlist)

label = list(df_train['label'])

# Create tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

# Rename for convienience
x_train = sequences
y_train = label

# Preprocess data
maxlen = 20
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)

In [31]:
# Create tf model
model = Sequential()
model.add(Embedding(100000, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=5,
                    batch_size=8,
                    validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             800000    
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 161       
Total params: 800,161
Trainable params: 800,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1661/1661 [==============================] - 5s 3ms/step - loss: 0.4821 - acc: 0.7769 - val_loss: 0.3662 - val_acc: 0.8272
Epoch 2/5
1661/1661 [==============================] - 4s 3ms/step - loss: 0.3043 - acc: 0.8634 - val_loss: 0.3256 - val_acc: 0.8504
Epoch 3/5
1661/1661 [==============================] - 4s 3ms/step - loss: 0.2610 - acc: 0.8873 - val_loss: 0.31

In [36]:
# Shuffle data to improve performance
df_test = shuffle(df_test)

# Create list from data
text = []

for index, rows in df_test.iterrows(): 
    wordlist = str(rows["text"])
    text.append(wordlist)

label = list(df_test['label'])
sequences = tokenizer.texts_to_sequences(text)

# Rename for convienience
x_test = sequences
y_test = label

# Preprocess data
maxlen = 20
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [38]:
model.evaluate(np.array(x_test), np.array(y_test))

130/130 [==============================] - 0s 434us/step - loss: 0.3157 - acc: 0.8613


[0.31567978858947754, 0.8613050580024719]

Evaluate

In [39]:
df_test_submission = pd.read_csv("../data/transformed/wrangled_test.csv")

In [43]:
# Create list from data
text = []

for index, rows in df_test_submission.iterrows(): 
    wordlist = str(rows["text"])
    text.append(wordlist)

sequences = tokenizer.texts_to_sequences(text)

# Rename for convienience
x_test_submission = sequences

# Preprocess data
maxlen = 20
x_test_submission = preprocessing.sequence.pad_sequences(x_test_submission, maxlen=maxlen)

In [44]:
preds = model.predict(np.array(x_test_submission),
    batch_size=8,
)

In [47]:
preds

array([[1.4201412e-01],
       [9.7554797e-01],
       [9.8831886e-01],
       ...,
       [3.0197409e-06],
       [1.0000000e+00],
       [2.7207077e-05]], dtype=float32)

In [48]:
df_test_submission_labels = pd.DataFrame(preds)

In [49]:
df_test_submission_labels.head()

,0
0,0.142014
1,0.975548
2,0.988319
3,0.229068
4,0.999954


In [51]:
df_test_submission["label"] = preds.tolist()

In [54]:
df_test_submission["label"] = df_test_submission["label"].apply(lambda x: round(x[0]))

In [55]:
df_test_submission.head()

,Unnamed: 0,id,title,author,text,text_length,nb_names,exclamation,label
0,0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",17,23,0,0
1,1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,4,1,0,1
2,2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,10,11,4,1
3,3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",8,10,0,0
4,4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,2,0,0,1


In [58]:
df_test_submission[["id", "label"]].to_csv("../data/kaggle/my_submit.csv", index=False)

Tokenizer + Regression